# Classification with supervised leanring

In [0]:
import numpy as np
import pandas as pd 
import warnings

In [0]:
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [0]:
#load dataset into pandas dataframe
df_news = pd.read_csv('https://raw.githubusercontent.com/ytwang3579/TextMining/master/dataset_0606.csv',encoding="utf-8")
data=df_news[df_news['source']=='focustaiwan']

In [0]:
data=data.drop_duplicates(subset=['title'], keep=False)

In [5]:
data

,date,title,content,url,source,section
1080,8/3/2020,Mets Taiwan Day 2020 to offer limited edition 'Taiwan No. 1' jersey,"New York, March 7 (CNA) The New York Mets organization will give away 500 Mets replica jerseys decorated with the name ""Taiwan"" and ""No. 1"" during the Major League Baseball (MLB) team's 16th annua...",https://focustaiwan.tw/sports/202003080006,focustaiwan,Sports
1081,9/3/2020,Taiwanese boxer secures ticket to Tokyo Olympics in Amman qualifier,"Taipei, March 9 (CNA) Taiwanese boxer Chen Nien-chin (陳念琴) on Sunday advanced to the semifinals of the Asia/Oceania Olympic Boxing Qualification event in Jordan, thereby earning a place in the 202...",https://focustaiwan.tw/sports/202003090007,focustaiwan,Sports
1082,9/3/2020,Second Taiwanese boxer secures ticket to Tokyo Olympics,"Taipei, March 9 (CNA) Taiwanese boxer Lin Yu-ting (林郁婷) beat 2016 Rio Olympics silver medalist Monday to advance to the semifinals of the Asia/Oceania Olympic Boxing Qualification event in Jordan,...",https://focustaiwan.tw/sports/202003090025,focustaiwan,Sports
1083,10/3/2020,Two more Taiwanese boxers qualify for Tokyo Olympics,"Taipei, March 10 (CNA) Taiwanese boxers Huang Hsiao-wen (黃筱雯) and Wu Shih-yi (吳詩儀) advanced to the semifinals of the Asia/Oceania Olympic Boxing Qualification event in Jordan on Monday, thereby ea...",https://focustaiwan.tw/sports/202003100010,focustaiwan,Sports
1084,14/3/2020,"Taiwan's Tai, Chou smash into semis at All England Badminton Open","Taipei, March 14 (CNA) Taiwanese badminton aces Tai Tzu-ying (戴資穎) and Chou Tien-chen (周天成) advanced into the semifinals of their respective categories at the Yonex All England Open after straight...",https://focustaiwan.tw/sports/202003140007,focustaiwan,Sports
...,...,...,...,...,...,...
2919,1/6/2020,Customers reminisce about bookstore where 'lights never went out',"Taipei, June 1 (CNA) A Taipei bookstore that had been open to customers around the clock since 1999 closed its doors on Sunday, but its special ambience and services will not be easily erased from...",https://focustaiwan.tw/culture/202006010013,focustaiwan,Culture
2920,2/6/2020,East Rift Valley Land Art Festival opens in Taitung,"Taipei, June 2 (CNA) The East Rift Valley Land Art Festival opened in Taitung County on Tuesday, featuring installations and other types of displays by local and foreign artists. The 22 artworks a...",https://focustaiwan.tw/culture/202006020022,focustaiwan,Culture
2921,3/6/2020,Documentary on Taiwanese food scores U.S. film prizes,"Taipei, June 3 (CNA) A short documentary focusing on Taiwanese food won two film awards in the United States recently, according to its producer, Deutsche Welle (DW). ""Tasty Taipei -- Food, Fusion...",https://focustaiwan.tw/culture/202006030012,focustaiwan,Culture
2922,4/6/2020,National Human Rights Museum wins award in Germany,"Taipei, June 4 (CNA) Taiwan's National Human Rights Museum (NHRM) won a ""Special Award"" at this year's Karl Wilhelm Fricke Award, for advancing education on human rights and democracy, the museum ...",https://focustaiwan.tw/culture/202006040013,focustaiwan,Culture


Definition POS of spacy here: https://spacy.io/api/annotation#pos-tagging


In [0]:
 # Define function for Lemmatization and feature selection using POS, spacy package
 #import spacy
 #def spacy_preprocess (text,lemma= True, pos= True, pos_select = ["VERB", "NOUN", "ADJ","ADV","PART"]):
   # Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
 #  nlp = spacy.load('en', disable=['parser', 'ner']) # disable parser, ner for faster loading
   # Parse the sentence using the loaded 'en' model object `nlp`
 #  doc = nlp(text)
 #  if pos== False:
 #    if lemma== True: text_preprocess= " ".join([token.lemma_ for token in doc])
 #    if lemma== False:text_preprocess= " ".join([token.text for token in doc])
 #  else:
 #    if lemma== True : text_preprocess= " ".join([token.lemma_ for token in doc if token.pos_ in pos_select])
 #    if lemma== False : text_preprocess= " ".join([token.text for token in doc if token.pos_ in pos_select])
 #  return text_preprocess

In [0]:
 #from IPython.display import clear_output
 #this may thake long time to process all 10.000 comments
 #data_lem_pos = data
 #for index in data_lem_pos.index:
 #   text =  data['content'][index]
 #   data_lem_pos['content'][index]= spacy_preprocess (text,lemma= True, pos= True, pos_select = ["VERB", "ADJ","ADV","NOUN"])
    # print out the perentage complete
 #   clear_output(wait=True)
 #   if ((index+1)%10==0):print("finish %s" %(str(round((index+1)/100,2))+"%"))

# # you should write the result after remove pos and lemmatizaiton to use later
#amazon_reviewDF_lem_pos.to_csv(mydrive+"Amazon_review_lem_pos_vaa.csv", index = False)

**Now you have 2 data frame:**

the original one:  *amazon_reviewDF*

the data after lemmatization, pos: *amazon_reviewDF_lem_pos*

**Split Training - Testing data for validation**

In [0]:
from sklearn.metrics import roc_auc_score
from sklearn import model_selection, metrics

def Prepare_Data(train_x, valid_x, train_y, valid_y,max_features = None):

  from sklearn import preprocessing
# encode the target variable, label Negative/Positive -> 0/1
  encoder = preprocessing.LabelEncoder()
  train_y_en = encoder.fit_transform(train_y)
  valid_y_en = encoder.fit_transform(valid_y)

  from sklearn.feature_extraction.text import TfidfVectorizer
# word level tf-idf
  tfidf_vect = TfidfVectorizer(stop_words='english',max_features=max_features)
  tfidf_vect.fit(train_x.values.astype('U')) #Learn vocabulary and idf from training data set. (in bad words)
  xtrain_tfidf =  tfidf_vect.transform(train_x.values.astype('U')) # compute tfidf for tranning data set
  xvalid_tfidf =  tfidf_vect.transform(valid_x.values.astype('U')) # compute tfidf for testing data set
  return xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en

Classified Tree

In [0]:
def Classified_Tree(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en):
  from sklearn import tree
  classifier_tree = tree.DecisionTreeClassifier(random_state=1)
  # fit the training dataset on the classifier
  classifier_tree =classifier_tree.fit(xtrain_tfidf, train_y_en)
  #print(train_y_en)
  #print(valid_y_en)
  # predict the labels on validation dataset
  valid_y_pred_tree = classifier_tree.predict(xvalid_tfidf)
  accuracy_tree = metrics.accuracy_score(valid_y_en, valid_y_pred_tree)
  #confusion_matrix_tree = metrics.confusion_matrix(valid_y_en, valid_y_pred_tree) 
  
  #confusion_matrix_tree = metrics.confusion_matrix(valid_y_en, valid_y_pred_tree, labels=[0,1,2,3,4,5,6])
  print("Classification Tree")
  print ("Accuracy", accuracy_tree)
  #F1_score1=metrics.f1_score(valid_y_en, valid_y_pred_tree)
  #print ("F-score", F1_score1)
  #print ("confusion_matrix\n", confusion_matrix_tree)
  #y_probas = classifier_tree.predict_proba(xvalid_tfidf)
  #print(roc_auc_score(valid_y_en, y_probas[0:,1]))

  y_prob = classifier_tree.predict_proba(xvalid_tfidf)

  macro_roc_auc_ovo = roc_auc_score(valid_y_en, y_prob, multi_class="ovo",average="macro")
  weighted_roc_auc_ovo = roc_auc_score(valid_y_en, y_prob, multi_class="ovo",average="weighted")
  macro_roc_auc_ovr = roc_auc_score(valid_y_en, y_prob, multi_class="ovr",average="macro")
  weighted_roc_auc_ovr = roc_auc_score(valid_y_en, y_prob, multi_class="ovr",average="weighted")
  print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} " "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
  print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} " "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr))

  print("\n")

**Use RandomForestClassifier- Full code**

In [0]:
def Random_Forest(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en):
  from sklearn import ensemble
  classifier_rf = ensemble.RandomForestClassifier(random_state= 1)
# fit the training dataset on the classifier
  classifier_rf.fit(xtrain_tfidf, train_y_en)

# predict the labels on validation dataset  
  valid_y_pred_rf = classifier_rf.predict(xvalid_tfidf)

  #Reverse factorize (converting y_pred from 0s,1s and 2s to Iris-setosa, Iris-versicolor and Iris-virginica
  #reversefactor = dict(zip(range(7),definitions))
  #valid_y_en = np.vectorize(reversefactor.get)(valid_y_en)
  #valid_y_pred_rf = np.vectorize(reversefactor.get)(valid_y_pred_rf)
  # Making the Confusion Matrix
  #print(pd.crosstab(valid_y_en, valid_y_pred_rf, rownames=['Actual Species'], colnames=['Predicted Species'])) 

  accuracy_rf = metrics.accuracy_score(valid_y_en, valid_y_pred_rf)
  #confusion_matrix_rf = metrics.confusion_matrix(valid_y_en, valid_y_pred_rf)
  #confusion_matrix_rf = metrics.confusion_matrix(valid_y_en, valid_y_pred_rf, labels=[0,1])
  #F1_score=metrics.f1_score(valid_y_en, valid_y_pred_rf)
  print("Random Forest")
  print ("Accuracy", accuracy_rf)
  #print ("F-score", F1_score)
  #print ("confusion_matrix\n", confusion_matrix_rf)
  y_prob = classifier_rf.predict_proba(xvalid_tfidf)

  macro_roc_auc_ovo = roc_auc_score(valid_y_en, y_prob, multi_class="ovo",average="macro")
  weighted_roc_auc_ovo = roc_auc_score(valid_y_en, y_prob, multi_class="ovo",average="weighted")
  macro_roc_auc_ovr = roc_auc_score(valid_y_en, y_prob, multi_class="ovr",average="macro")
  weighted_roc_auc_ovr = roc_auc_score(valid_y_en, y_prob, multi_class="ovr",average="weighted")
  print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} " "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
  print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} " "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr))


  print("\n")

**Use Neural Network- Multi-layer Perceptron classifier.**

In [0]:
def Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,hidden_layer_sizes=(100, ),activation='relu'):
  from sklearn.neural_network import MLPClassifier
  classifier_nn = MLPClassifier(random_state=1,hidden_layer_sizes=hidden_layer_sizes,activation=activation)
# fit the training dataset on the classifier
  classifier_nn.fit(xtrain_tfidf, train_y_en) # take around 3-4 minutes

# predict the labels on validation dataset
  valid_y_pred_nn = classifier_nn.predict(xvalid_tfidf)

  accuracy_nn = metrics.accuracy_score(valid_y_en, valid_y_pred_nn)
  #confusion_matrix_nn = metrics.confusion_matrix(valid_y_en, valid_y_pred_nn, labels=[0,1])
  print("Neural Network")
  print ("Accuracy", accuracy_nn)
  #F1_score=metrics.f1_score(valid_y_en, valid_y_pred_nn)
  #print ("F-score", F1_score)
  #print ("confusion_matrix\n", confusion_matrix_nn)
  
  y_prob = classifier_nn.predict_proba(xvalid_tfidf)

  macro_roc_auc_ovo = roc_auc_score(valid_y_en, y_prob, multi_class="ovo",average="macro")
  weighted_roc_auc_ovo = roc_auc_score(valid_y_en, y_prob, multi_class="ovo",average="weighted")
  macro_roc_auc_ovr = roc_auc_score(valid_y_en, y_prob, multi_class="ovr",average="macro")
  weighted_roc_auc_ovr = roc_auc_score(valid_y_en, y_prob, multi_class="ovr",average="weighted")
  print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} " "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
  print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} " "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr))


  print("\n")
  

Logistic Regression

In [0]:
def Logistic_Regression(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en):
  from sklearn.linear_model import LogisticRegression
  classifier_lr = LogisticRegression(random_state=1)
# fit the training dataset on the classifier
  classifier_lr.fit(xtrain_tfidf, train_y_en) # take around 3-4 minutes

# predict the labels on validation dataset
  valid_y_pred_lr = classifier_lr.predict(xvalid_tfidf)

  accuracy_lr = metrics.accuracy_score(valid_y_en, valid_y_pred_lr)
  #confusion_matrix_lr = metrics.confusion_matrix(valid_y_en, valid_y_pred_lr, labels=[0,1])
  print("Logistic Regression")
  print ("Accuracy", accuracy_lr)
  #F1_score=metrics.f1_score(valid_y_en, valid_y_pred_lr)
 # print ("F-score", F1_score)
  #print ("confusion_matrix\n", confusion_matrix_lr)

  y_prob = classifier_lr.predict_proba(xvalid_tfidf)

  macro_roc_auc_ovo = roc_auc_score(valid_y_en, y_prob, multi_class="ovo",average="macro")
  weighted_roc_auc_ovo = roc_auc_score(valid_y_en, y_prob, multi_class="ovo",average="weighted")
  macro_roc_auc_ovr = roc_auc_score(valid_y_en, y_prob, multi_class="ovr",average="macro")
  weighted_roc_auc_ovr = roc_auc_score(valid_y_en, y_prob, multi_class="ovr",average="weighted")
  print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} " "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
  print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} " "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr))


  print("\n")

No Lemmatization & POS

In [13]:
#No Lemmatization & POS
#split the dataset into training and testing 
#test_size = 0.1 mean 90% data is used for traning data, 10% for testing data.
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(data['content'], 
                                                      data['section'],test_size=0.2,random_state=1)

xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en = Prepare_Data(train_x, valid_x, train_y, valid_y)
print("No Lemmatization & POS\n")

Classified_Tree(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en)
Random_Forest(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en)
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en)
Logistic_Regression(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en)

No Lemmatization & POS

Classification Tree
Accuracy 0.6696969696969697
One-vs-One ROC AUC scores:
0.718483 (macro),
0.742728 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.726565 (macro),
0.775053 (weighted by prevalence)


Random Forest
Accuracy 0.8121212121212121
One-vs-One ROC AUC scores:
0.949417 (macro),
0.956732 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.964924 (macro),
0.965930 (weighted by prevalence)


Neural Network
Accuracy 0.8545454545454545
One-vs-One ROC AUC scores:
0.963536 (macro),
0.968403 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.975634 (macro),
0.970821 (weighted by prevalence)


Logistic Regression
Accuracy 0.806060606060606
One-vs-One ROC AUC scores:
0.954395 (macro),
0.963591 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.971710 (macro),
0.969355 (weighted by prevalence)




In [14]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(100, 100))

Neural Network
Accuracy 0.8515151515151516
One-vs-One ROC AUC scores:
0.959000 (macro),
0.967430 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.975198 (macro),
0.971065 (weighted by prevalence)




In [15]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(128, 128))

Neural Network
Accuracy 0.8636363636363636
One-vs-One ROC AUC scores:
0.961953 (macro),
0.968157 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.975958 (macro),
0.971247 (weighted by prevalence)




In [16]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,hidden_layer_sizes=(100, 100,100))

Neural Network
Accuracy 0.8575757575757575
One-vs-One ROC AUC scores:
0.958316 (macro),
0.967586 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.974727 (macro),
0.971564 (weighted by prevalence)




In [17]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(200))

Neural Network
Accuracy 0.8606060606060606
One-vs-One ROC AUC scores:
0.961435 (macro),
0.967908 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.974918 (macro),
0.970545 (weighted by prevalence)




In [18]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(50))

Neural Network
Accuracy 0.8666666666666667
One-vs-One ROC AUC scores:
0.960251 (macro),
0.967724 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.974893 (macro),
0.970712 (weighted by prevalence)




In [19]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(200,200))

Neural Network
Accuracy 0.8575757575757575
One-vs-One ROC AUC scores:
0.964523 (macro),
0.968743 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.976166 (macro),
0.971069 (weighted by prevalence)




In [20]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(200,200,200))

Neural Network
Accuracy 0.8484848484848485
One-vs-One ROC AUC scores:
0.953494 (macro),
0.964606 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.971189 (macro),
0.969341 (weighted by prevalence)




In [21]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(256,256))

Neural Network
Accuracy 0.8515151515151516
One-vs-One ROC AUC scores:
0.962678 (macro),
0.967987 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.975500 (macro),
0.971197 (weighted by prevalence)




In [22]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(256,256,128))

Neural Network
Accuracy 0.8575757575757575
One-vs-One ROC AUC scores:
0.964405 (macro),
0.968954 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.976592 (macro),
0.971135 (weighted by prevalence)




In [23]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(256,128))

Neural Network
Accuracy 0.8545454545454545
One-vs-One ROC AUC scores:
0.963254 (macro),
0.968438 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.975619 (macro),
0.971320 (weighted by prevalence)




In [0]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(200,100))

In [0]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(200,100,50))

In [0]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(50,50))

In [0]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(128,64))

In [0]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,(128,64,64))

In [0]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,activation="identity")

In [0]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,activation="logistic")

In [0]:
Neural_Network(xtrain_tfidf,train_y_en,xvalid_tfidf,valid_y_en,activation="tanh")